In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
from rasterio.plot import show
from rasterio.windows import Window

In [2]:
tiff5 = './assets/DHMVIIDSMRAS1m_k05.tif'
tiff12 = "./assets/DHMVIIDSMRAS1m_k12.tif"
tiff13 = "./assets/DHMVIIDSMRAS1m_k13.tif"


In [3]:
DHM5 = gpd.read_file('./assets/DHMVIIDTMRAS1m_k05/DHMVII_vdc_k5.shp')

## Geopandas tryout

In [4]:
DHM5.head()

,Date,OpnDatum1,OpnDatum2,geometry
0,20140319,2014-03-19,None,"POLYGON Z ((86000.000 230206.366 0.000, 85998...."
1,20140317,2014-03-19,2014-03-17,"POLYGON Z ((86000.000 230206.366 0.000, 86000...."
2,20140317,2014-03-17,None,"MULTIPOLYGON Z (((97000.000 228000.000 0.000, ..."
3,20140317,2014-03-17,2014-03-30,"MULTIPOLYGON Z (((95075.940 227675.980 0.000, ..."
4,20140330,2014-03-30,None,"POLYGON Z ((87000.000 227360.941 0.000, 87000...."


In [5]:
type(DHM5)

geopandas.geodataframe.GeoDataFrame

In [6]:
plot(DHM5)

NameError: name 'plot' is not defined

## RasterIO

## CREATE RASTERIO DATASET OBJECT

In [7]:
data12 = rasterio.open(tiff12)

# show(data12)



## Get metadata

In [8]:
data12.meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -9999.0,
 'width': 32000,
 'height': 20000,
 'count': 1,
 'crs': CRS.from_epsg(31370),
 'transform': Affine(1.0, 0.0, 34000.0,
        0.0, -1.0, 218000.0)}

## Get raterio dataset's bounds

In [9]:
data12.bounds

BoundingBox(left=34000.0, bottom=198000.0, right=66000.0, top=218000.0)

In [ ]:
# rasterio.plot.show(data12,cmap='terrain')

In [10]:
type(data12)

rasterio.io.DatasetReader

In [ ]:
with rasterio.open(data12) as src:
    w = src.read(1, window=Window(0, 0, 512, 256))

In [ ]:
# data5 = rasterio.open(tiff5)

# show(data5)

In [ ]:
# data13 = rasterio.open(tiff13)

# show(data13)

In [ ]:
# data13.meta

## GDAL

In [ ]:
# from osgeo import gdal
# import matplotlib.pyplot as plt

# dataset = gdal.Open(tiff12, gdal.GA_ReadOnly) # Note GetRasterBand() takes band no. starting from 1 not 0
# band = dataset.GetRasterBand(1)
# arr = band.ReadAsArray()
# plt.imshow(arr)